## VGG_BPCA
Here we are using a VGG net with 19 layers with the dataset cifar 100.

We are trying to reproduce the results of the paper [1] changing the max pooling layers for blocked based principal component analysis (BPCA) layers like the article [2] proposes.

[1] https://arxiv.org/pdf/1706.05350.pdf

[2] http://www.ic.uff.br/iwssip2010/Proceedings/nav/papers/paper_10.pdf

[3] https://ieeexplore.ieee.org/document/5654484